# CIFAR-10 CNN Training with Advanced Convolutions 🚀\n
\n
This notebook demonstrates training a CNN with:\n
- Depthwise Separable Convolution\n
- Dilated Convolution\n
- Global Average Pooling\n
- Target: 85% accuracy, < 200k parameters

In [ ]:
# Core imports\n
import torch\n
import torch.optim as optim\n
import torch.nn as nn\n
import numpy as np\n
import matplotlib.pyplot as plt\n
from torchsummary import summary\n
\n
# Set device\n
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n
print(f"Using device: {device}")

In [ ]:
# Import modules\n
from models.cifar10_net import CIFAR10Net\n
from models.trainer import Trainer\n
from models.tester import Tester\n
from data.dataset import CIFAR10DataLoader\n
from utils.visualization import TrainerVisualizer\n
from config import Config\n
\n
# Initialize components\n
config = Config()\n
trainer = Trainer()\n
tester = Tester()\n
visualizer = TrainerVisualizer()\n
\n
print("✅ Modules imported successfully!")

In [ ]:
# Create and analyze model\n
model = CIFAR10Net(dropout_value=config.MODEL_DROPOUT)\n
model = model.to(device)\n
\n
# Print model summary\n
print("Model Architecture:")\n
summary(model, input_size=(3, 32, 32))\n
\n
# Parameter analysis\n
param_count = model.count_parameters()\n
print(f"\nParameters: {param_count:,}")\n
print(f"Under 200k: {'✅' if param_count < 200000 else '❌'}")

In [ ]:
# Load data\n
data_loader = CIFAR10DataLoader(batch_size=config.DEFAULT_BATCH_SIZE)\n
train_loader, test_loader, classes, norm_params = data_loader.get_data()\n
\n
print(f"Dataset loaded - Train: {len(train_loader.dataset):,}, Test: {len(test_loader.dataset):,}")\n
print(f"Classes: {classes}")

In [ ]:
# Setup training\n
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)\n
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)\n
\n
print(f"Optimizer: SGD(lr=0.1)\])\n
print(f"Epochs: {config.DEFAULT_EPOCHS}")

In [ ]:
# Training loop\n
train_losses = []\n
test_losses = []\n
train_accuracies = []\n
test_accuracies = []\n
\n
print(f"🚀 Starting training for {config.DEFAULT_EPOCHS} epochs")\n
\n
for epoch in range(config.DEFAULT_EPOCHS):\n
    print(f"\nEpoch {epoch + 1}/{config.DEFAULT_EPOCHS}")\n
    \n
    # Training\n
    train_results = trainer.train(model, device, train_loader, optimizer, epoch)\n
    train_losses.extend(train_results['losses'])\n
    train_accuracies.extend(train_results['accuracies'])\n
    \n
    # Testing\n
    test_results = tester.test(model, device, test_loader)\n
    test_losses.append(test_results['loss'])\n
    test_accuracies.append(test_results['accuracy'])\n
    \n
    # Learning rate scheduling\n
    scheduler.step()\n
    \n
    print(f"Train Acc: {train_results['final_accuracy']:.2f}%, Test Acc: {test_results['accuracy']:.2f}%")\n
\n
print("Training completed!")

In [ ]:
# Plot results\n
visualizer.plot_training_curves(train_losses, train_accuracies, test_losses, test_accuracies)

In [ ]:
# Generate report\n
report = tester.generate_classification_report(model, test_loader, classes, device)\n
print(f"Final Accuracy: {report['accuracy']:.2%}")\n
print(f"Parameters: {model.count_parameters():,}")

In [ ]:
# Save model\n
torch.save({\n
    'model_state_dict': model.state_dict(),\n
    'model_class': CIFAR10Net,\n
    'classes': classes,\n
    'final_accuracy': report['accuracy']\n
}, 'cifar10_model.pth')\n
\n
print("Model saved successfully!")

## Data Loading and Training Preparation 📊

In [ ]:
# Load CIFAR-10 data with Albumentations
data_loader = CIFAR10DataLoader(batch_size=config.DEFAULT_BATCH_SIZE)
train_loader, test_loader, classes, norm_params = data_loader.get_data()

print(f"Dataset: {len(train_loader.dataset):,} train, {len(test_loader.dataset):,} test")
print(f"Classes: {classes}")

In [ ]:
# Visualize sample images
visualizer.show_sample_images(train_loader, classes, undo_normalization=norm_params, num_images=12)

## Training Setup 🚀

In [ ]:
# Setup optimizer and scheduler
optimizer = optim.SGD(model.parameters(), lr=config.DEFAULT_LEARNING_RATE, momentum=config.DEFAULT_MOMENTUM, weight_decay=config.DEFAULT_WEIGHT_DECAY)
scheduler = StepLR(optimizer, step_size=config.LR_SCHEDULER_STEP_SIZE, gamma=config.LR_SCHEDULER_GAMMA)

print("Training Configuration:")
print(f"Optimizer: SGD with lr={config.DEFAULT_LEARNING_RATE}")
print(f"Epochs: {config.DEFAULT_EPS}")
print(f"Target: {config.TARGET_ACCURACY}% accuracy")